1. Træne en model til at kunne gennemskue, om et review er positivt eller negativt baseret på tekstindholdet (classification & natural language processing)
2. Gruppere mest populære ord i positive og negative reviews, og visualisere det
3. Sammenligne den trænede model mod det rigtige datasæt, for at se hvor nøjagtig den er ift. Datasættet
4. Se om der er en sammenhæng mellem længde af review og rating
5. Fordi datasættets format er simpel, kan vi webscrape reviews fra andre sider, og bruge vores trænede model på dem for at se, om den kan genneskue
Om reviewet er positivt eller negativt.


In [ ]:
pip install fasttext termcolor wget

### 1. Træne en model til at kunne gennemskue, om et review er positivt eller negativt baseret på tekstindholdet (classification & natural language processing)

In [ ]:
import model
model.download_dataset()

model.extract_dataset()

model.clean_dataset()

parameter: seconds
model.train_autotune(600)

model.test_model("trained_review_model.bin")

model.predict_with_model(
    [
        ("The ps5 dual sense controller is comfortable its a mix between an Xbox elite series 2 controller and a ps4 dualshock controller i got it to work on pc but sadly can't test out rumble features", 5),
        ("Controller came in today. It does not come with any USB-C cable, which is a bummer, but not unexpected. I could not get it to work on my PS4 via wired connection, but it does work for some reason on the PS3. Was able to play some PS3 games with the controller (not all games work), only thing that fails to function on the PS3 is the home button and the vibration. Feels comfortable to play on and fits right in the palms of my hands. Looking forward to using it on an actual PS5 :)", 4),
        ("The process of buying the vehicle was super smooth and easy. On that I have no complaints. My complaint is two fold, I sold them a vehicle that had brand new tires and an oil change. The vehicle I bought is now sitting at 45% oil life, so they didnt change the oil that is lame, every vehicle that is being sold shouldve had an oil change prior to being sold. The tire issue is the hardest and I wasn't going to say a thing until I seen the oil life. So the tires on the vehicle I bought have about 25 to 30% tread life left, the vehicle I sold them had 95%+.", 3),
        ("Great idea, but super slow. Trying to buy a car was a long painful process. They are constantly on their iPad trying to communicate with other people. Took forrreeevvvver.", 2),
        ("The controller is great!! Feels great and is loaded with cool new features...but then you have Sony customer service. A button started to malfunction on the controller that was about a month old. I called Sony up and instead of apologizing profusely and sending a prepaid postage to get it fixed or just replacing it, I got an email saying I’m responsible for shipping costs. Way to stand by your product!!! You made a device that malfunctioned...why not put a burden on the consumer to ship your crap back to you. Bad, bad Sony. I’m buying an XBOX.", 1),
    ], "trained_review_model.bin")

model.get_model_args("trained_review_model.bin")

### Sammenligne den trænede model mod det rigtige datasæt, for at se hvor nøjagtig den er ift. Datasættet

In [ ]:
import model
model.download_model_with_085_precision()

model.test_model("trained_review_stars_model.0.85.bin")

model.predict_with_model(
    [
        ("The ps5 dual sense controller is comfortable its a mix between an Xbox elite series 2 controller and a ps4 dualshock controller i got it to work on pc but sadly can't test out rumble features", 5),
        ("Controller came in today. It does not come with any USB-C cable, which is a bummer, but not unexpected. I could not get it to work on my PS4 via wired connection, but it does work for some reason on the PS3. Was able to play some PS3 games with the controller (not all games work), only thing that fails to function on the PS3 is the home button and the vibration. Feels comfortable to play on and fits right in the palms of my hands. Looking forward to using it on an actual PS5 :)", 4),
        ("The process of buying the vehicle was super smooth and easy. On that I have no complaints. My complaint is two fold, I sold them a vehicle that had brand new tires and an oil change. The vehicle I bought is now sitting at 45% oil life, so they didnt change the oil that is lame, every vehicle that is being sold shouldve had an oil change prior to being sold. The tire issue is the hardest and I wasn't going to say a thing until I seen the oil life. So the tires on the vehicle I bought have about 25 to 30% tread life left, the vehicle I sold them had 95%+.", 3),
        ("Great idea, but super slow. Trying to buy a car was a long painful process. They are constantly on their iPad trying to communicate with other people. Took forrreeevvvver.", 2),
        ("The controller is great!! Feels great and is loaded with cool new features...but then you have Sony customer service. A button started to malfunction on the controller that was about a month old. I called Sony up and instead of apologizing profusely and sending a prepaid postage to get it fixed or just replacing it, I got an email saying I’m responsible for shipping costs. Way to stand by your product!!! You made a device that malfunctioned...why not put a burden on the consumer to ship your crap back to you. Bad, bad Sony. I’m buying an XBOX.", 1),
    ], "trained_review_stars_model.0.85.bin")

In [ ]:
model.get_model_args("trained_review_stars_model.0.85.bin")